# Creating Remote Servers in the Cloud



There are many different cloud computing providers, such as Amazon AWS, Google Cloud Platform, IBM Bluemix, DigitalOcean, etc.

In this lecture we will use the latter.

![](https://www.digitalocean.com/assets/media/homepage/create-db9113c3.gif)

If you never setup a remote server with DigitalOcean (DO), then try it now following [this tutorial](https://www.digitalocean.com/community/tutorials/how-to-create-your-first-digitalocean-droplet)

To log onto your remote servers more easily and properly you setup a pair of SSH keys and register the public key woth DigitalOcean. See [this tutorial](https://www.digitalocean.com/community/tutorials/how-to-set-up-ssh-keys--2) for how to do this.

That is it. After a few clicks you have your own server up and running.


# Your Turn!

![](https://camo.githubusercontent.com/320b4791da998fd94e34ad4a85d44d8d5a581ca4/68747470733a2f2f732d6d656469612d63616368652d616b302e70696e696d672e636f6d2f6f726967696e616c732f39662f37332f65332f39663733653366386139353864626530336230663736663838313161353461312e676966)

  * Setup a server at DigitalOcean with the latest version of Ubuntu Linux.
  * If you do not have already a pair of SSH keys, create one now.
  * Choose the smallest doplet (for $5/month).
  * Register your public SSH key.
  * Log onto the newly created remote server from your commandline via SSH.
  * Execute some Bash commands, see last session, on the remote machine to see that everything is working as expected.
  * When you are done experimenting destroy the remote server (destroy the droplet in DOs homepage).
  
Note, droplets are billed per hour. Assuming that your experiment takes two to three hours, youl will be billed ca 2 cents.

# A Small Detour - Deployment with Vagrant Provisioners

The last time you build two VMs one with a web-aaplication and one with a database engine. You were running those VMs on you local machine. In the best sense of describing and storing _infrastructure as code_, Vagrant allows you to 'plug-in' various provisioners, so that you can steer remote machines instead of local VMs.

For example, you can deploy the application from the last session on two remote servers in the cloud -here _droplets at DigitalOcean_- just by running `vagrant up`. 

In case you do not have the DigitalOcean Vagrant Provider installed do so by running:

```bash
$ vagrant plugin install vagrant-digitalocean
```

See the documentation at https://github.com/devopsgroup-io/vagrant-digitalocean for more details.

Now given that you saved the contents given below in a `Vagrantfile`, you can create two droplets with a web-server and database engine by running:


```bash
$ vagrant up --provider=digital_ocean
```



```ruby
# -*- mode: ruby -*-
# vi: set ft=ruby :

Vagrant.configure("2") do |config|
  config.vm.box = 'digital_ocean'
  config.vm.box_url = "https://github.com/devopsgroup-io/vagrant-digitalocean/raw/master/box/digital_ocean.box"
  config.ssh.private_key_path = '~/.ssh/id_rsa'
  
  config.vm.define "webserver", primary: true do |server|

    server.vm.provider :digital_ocean do |provider|
      provider.ssh_key_name = ENV["SSH_KEY_NAME"]
      provider.token = ENV["DIGITAL_OCEAN_TOKEN"]
      provider.image = 'ubuntu-16-04-x64'
      provider.region = 'fra1'
      provider.size = '512mb'
      provider.privatenetworking = true
    end

    server.vm.hostname = "webserver"
    server.vm.provision "shell", inline: <<-SHELL
      echo "Installing Go Lang"
      wget https://storage.googleapis.com/golang/go1.8.3.linux-amd64.tar.gz
      sudo tar -C /usr/local -xzf go1.8.3.linux-amd64.tar.gz
      rm go1.8.3.linux-amd64.tar.gz

      mkdir -p $HOME/go/src/github.com/HelgeCPH/mywebserver/
      mkdir -p $HOME/go/bin
      mkdir -p $HOME/go/pkg

      echo "export PATH=$PATH:/usr/local/go/bin" >> $HOME/.profile
      echo "export GOPATH=/go_projects" >> $HOME/.profile

      export PATH=$PATH:/usr/local/go/bin
      export GOPATH=/go_projects

      go get gopkg.in/mgo.v2

      cp /vagrant/main.go $HOME/go/src/github.com/HelgeCPH/mywebserver/telbookserver.go
      
      go build $HOME/go/src/github.com/HelgeCPH/mywebserver/telbookserver.go
      ./telbookserver &
    echo "==================================================================="
    echo "=                             DONE                                ="
    echo "==================================================================="
    echo "To log onto the VM:"
    echo "$ vagrant ssh"
    echo "To start the webserver:"
    echo "$ ./telbookserver > /tmp/out.log 2>&1 &"
    SHELL
  end

  config.vm.define "dbserver", primary: false do |server|
    server.vm.provider :digital_ocean do |provider|
      provider.ssh_key_name = ENV["SSH_KEY_NAME"]
      provider.token = ENV["DIGITAL_OCEAN_TOKEN"]
      provider.image = 'ubuntu-16-04-x64'
      provider.region = 'fra1'
      provider.size = '512mb'
      provider.privatenetworking = true
    end

    server.vm.hostname = "dbserver"
    server.vm.provision "shell", inline: <<-SHELL
      echo "Installing MongoDB"
      sudo apt-get -y install mongodb-server
      sudo mkdir -p /data/db
    
      mongo /vagrant/db_setup.js

      sudo sed -i '/bind_ip = / s/127.0.0.1/0.0.0.0/' /etc/mongodb.conf
      sudo service mongodb restart
    SHELL
  end

  config.vm.provision "shell", privileged: false, inline: <<-SHELL
    sudo apt-get update
  SHELL
end
```

To make this work properly, you have to have a pair of SSH keys, and they are lying in your home directory in a hidden directory, e.g., `~/.ssh/id_rsa`. In case you have them stored somewhere else, you have to modify the line `config.ssh.private_key_path = '~/.ssh/id_rsa'`accordingly.

Additionally, you have to have two environment variables set. One `$SSH_KEY_NAME` which is the name of the key that you have registered at DO's homepage. Furthermore, the `Vagrantfile` assumes that you have set the environment variable `$DIGITAL_OCEAN_TOKEN` to the API token, which you receive from DO, see [this] tutorial(https://www.digitalocean.com/community/tutorials/how-to-use-the-digitalocean-api-v2)

# Your Turn!
![](https://camo.githubusercontent.com/320b4791da998fd94e34ad4a85d44d8d5a581ca4/68747470733a2f2f732d6d656469612d63616368652d616b302e70696e696d672e636f6d2f6f726967696e616c732f39662f37332f65332f39663733653366386139353864626530336230663736663838313161353461312e676966)

  * Create a `Vagrantfile` that creates the smallest droplet at DO running an Apache webserver.
  * In your provision script create an `index.html` file, which just displays a "Hello, world!" message.
  * Point your browser to http://<your_ip> to check that your homepage is served correctly.
  * Destroy your droplet by running `vagrant destroy` after you are done.
  * Check with `vagrant global-status` that you remote machine was destroyed properly. You might want to run `vagrant global-status --prune` to verify that the remote machine was really destroyed.